## Visualizing Spatial Data with Pandas and Bokeh

[bokeh](http://bokeh.pydata.org/en/latest/) is a relatively new JavaScript visualization language for Python that is modeled after D3 but is intended to be able to handle millions of data points.

>Bokeh is a Python interactive visualization library that targets modern web browsers for presentation. Its goal is to provide elegant, concise construction of novel graphics in the style of D3.js, and to extend this capability with high-performance interactivity over very large or streaming datasets. Bokeh can help anyone who would like to quickly and easily create interactive plots, dashboards, and data applications. ([Bokeh Website](http://bokeh.pydata.org/en/latest/))

The advantage of Bokeh over matplotlib is that the visualizations can be interactive (via the JavaScript).

From the U.K. accident data, we can plot the location of accidents for which latitude and longitude values are provided.

In [1]:
!pip install bokeh=='0.12.16'

     |████████████████████████████████| 14.7MB 130kB/s  eta 0:00:01   |██▍                             | 1.1MB 971kB/s eta 0:00:14     |███████████████▋                | 7.1MB 971kB/s eta 0:00:08
  Stored in directory: /home/gastonq/.cache/pip/wheels/ff/28/51/22e8d08e9d5383ee1de981aaa8ff7bc53c7d65022e5101400f
Successfully built bokeh
  Found existing installation: bokeh 1.0.4
    Uninstalling bokeh-1.0.4:
      Successfully uninstalled bokeh-1.0.4


In [3]:
import os
import sqlite3 as sqlite
DATADIR = os.path.join(os.path.expanduser("~"),"DATA",
                       "Misc")
print(os.path.exists(DATADIR))
import pandas as pd
import numpy as np

True


In [4]:

from bokeh.io import output_notebook

### This enables drawing directly in the notebook

In [5]:
output_notebook()

Loading BokehJS ...

### Read in the data

In [21]:
data = pd.read_csv(os.path.join(DATADIR,
                         "Accidents7904.csv"),nrows=5000,
                   usecols = ['Longitude',"Latitude","Date","Number_of_Casualties"])#.dropna()
data.head()

,Longitude,Latitude,Number_of_Casualties,Date
0,NaN,NaN,1,18/01/1979
1,NaN,NaN,1,01/01/1979
2,NaN,NaN,3,01/01/1979
3,NaN,NaN,2,01/01/1979
4,NaN,NaN,1,01/01/1979


In [14]:
data.describe()
#data.dtypes

,Location_Easting_OSGR,Location_Northing_OSGR,Longitude,Latitude,Police_Force,Accident_Severity,Number_of_Vehicles,Number_of_Casualties,Day_of_Week,Local_Authority_(District),...,Pedestrian_Crossing-Human_Control,Pedestrian_Crossing-Physical_Facilities,Light_Conditions,Weather_Conditions,Road_Surface_Conditions,Special_Conditions_at_Site,Carriageway_Hazards,Urban_or_Rural_Area,Did_Police_Officer_Attend_Scene_of_Accident,LSOA_of_Accident_Location
count,997.000000,997.000000,0.0,0.0,1000.0,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,...,1000.000000,1000.000000,1000.000000,1000.000000,1000.00000,1000.0,1000.000000,1000.0,1000.0,0.0
mean,294988.335005,501388.164493,NaN,NaN,1.0,2.859000,1.730000,1.223000,3.937000,30.379000,...,-0.905000,-0.644000,2.514000,6.874000,1.93900,-1.0,0.010000,-1.0,-1.0,NaN
std,108430.874548,291362.859755,NaN,NaN,0.0,0.375846,0.737322,0.565323,1.682219,80.495583,...,0.293362,1.241292,1.518588,2.309428,0.78479,0.0,0.161015,0.0,0.0,NaN
min,44670.000000,1000.000000,NaN,NaN,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,...,-1.000000,-1.000000,1.000000,2.000000,1.00000,-1.0,-1.000000,-1.0,-1.0,NaN
25%,226570.000000,240000.000000,NaN,NaN,1.0,3.000000,1.000000,1.000000,3.000000,8.000000,...,-1.000000,-1.000000,1.000000,8.000000,1.00000,-1.0,0.000000,-1.0,-1.0,NaN
50%,294080.000000,514000.000000,NaN,NaN,1.0,3.000000,2.000000,1.000000,4.000000,17.000000,...,-1.000000,-1.000000,4.000000,8.000000,2.00000,-1.0,0.000000,-1.0,-1.0,NaN
75%,360870.000000,762000.000000,NaN,NaN,1.0,3.000000,2.000000,1.000000,5.000000,26.000000,...,-1.000000,-1.000000,4.000000,8.000000,3.00000,-1.0,0.000000,-1.0,-1.0,NaN
max,572080.000000,993000.000000,NaN,NaN,1.0,3.000000,5.000000,5.000000,7.000000,513.000000,...,0.000000,5.000000,6.000000,8.000000,3.00000,-1.0,2.000000,-1.0,-1.0,NaN


####  We can use the [``sample``](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.sample.html) method to get a subset of DataFrame

In [24]:
subdata = data.sample(2000)
mean_long = np.mean(subdata['Longitude'])
mean_lat  = np.mean(subdata['Latitude'])


In [30]:
from bokeh.io import output_file, show
from bokeh.models import (
  GMapPlot, GMapOptions, ColumnDataSource, Circle, DataRange1d, 
    PanTool, WheelZoomTool, BoxSelectTool, HoverTool
)

hover = HoverTool()
map_options = GMapOptions(lat=mean_lat, 
                          lng=mean_long, 
                          map_type="roadmap", zoom=6)

plot = GMapPlot(
    x_range=Range1d(), 
 y_range=Range1d(), 
    map_options=map_options
)
plot.title.text = "U.K. Road Accidents"

source = ColumnDataSource(
    data=dict(
        lat=subdata['Latitude'],
        lon=subdata['Longitude'],
    )
)


hover.tooltips.append(('index','$index'))
circle = Circle(x="lon", y="lat", size=2, 
                fill_color="blue", fill_alpha=0.8, 
                line_color=None)
plot.add_glyph(source, circle)

plot.add_tools(PanTool(),WheelZoomTool(), BoxSelectTool(), hover)
show(plot)

NameError: name 'Range1d' is not defined